In [1]:
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import tokenizers
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

from parameters import *

In [2]:
train = pd.read_csv(TRAIN_DIR).dropna()
texts = train['full_text'].to_numpy()

In [17]:
def embedding(texts):
    tokenizer = AutoTokenizer.from_pretrained(DEBERTA)
    tokenizer.save_pretrained(DEBERTA_DIR)
    
    encodings = [tokenizer.encode_plus(
        text, 
        max_length=MAX_LENGTH,
        return_tensors='pt', 
        add_special_tokens=True, 
        truncation=True,
        padding=True
    ) for text in tqdm(texts)]
    return encodings

In [18]:
tokenizer = AutoTokenizer.from_pretrained(DEBERTA)
tokenizer.save_pretrained(DEBERTA_DIR)

encodings = embedding(texts)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 3911/3911 [00:05<00:00, 654.05it/s]


In [20]:
class model(nn.Module):
    def __init__(self):
        self.config = AutoConfig.from_pretrained(DEBERTA, output_hidden_states=True)
        self.model = AutoModel(self.config)
        self.fc = nn.Linear(self.config.hidden_size, 6)
        
    def _mean_pooling(self, outputs, attention_mask):
        input_mask = attention_mask.unsqueeze(-1).expand(outputs.size()).float()
        return torch.sum(outputs * input_mask, 1) / torch.clamp(input.sum(1), min=1e-9)
    
    def forward(self, inputs):
        outputs = self.model(**inputs)
        outputs = self._mean_pooling(outputs[0], inputs['attention_mask'])
        return self.fc(outputs)